# Extracting Features with ViT
- https://huggingface.co/docs/transformers/model_doc/vit#vision-transformer-vit
- https://arxiv.org/abs/2010.11929

In [21]:
# import os
import torch
# import csv

import pandas as pd
import numpy as np
# import torchvision.transforms as transforms

from PIL import Image
from transformers import ViTImageProcessor, ViTFeatureExtractor, ViTModel
from sklearn.model_selection import train_test_split, GridSearchCV

2023-04-25 05:26:51.994421: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
BASE = '/Users/brinkley97/Documents/development/'
CLASS_PATH = 'classes/csci_535_multimodal_probabilistic_learning/'
DATASET_PATH = 'datasets/'

In [23]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [7]:
def load_data(file):
    original_data = pd.read_csv(file)
    # original_data = pd.DataFrame(file)
    copy_of_data = original_data.copy()
    return copy_of_data

In [8]:
file_paths = BASE + CLASS_PATH + DATASET_PATH + 'cremaD_mspR_mspT.csv'

In [14]:
dataset_paths_copy = load_data(file_paths)
dataset_paths_copy

Unnamed: 0                                      CREMA-D Paths  \
0                0  /Users/brinkley97/Documents/development/classe...   
1                1  /Users/brinkley97/Documents/development/classe...   
2                2  /Users/brinkley97/Documents/development/classe...   
3                3  /Users/brinkley97/Documents/development/classe...   
4                4  /Users/brinkley97/Documents/development/classe...   
...            ...                                                ...   
114818      114818  /Users/brinkley97/Documents/development/classe...   
114819      114819  /Users/brinkley97/Documents/development/classe...   
114820      114820  /Users/brinkley97/Documents/development/classe...   
114821      114821  /Users/brinkley97/Documents/development/classe...   
114822      114822  /Users/brinkley97/Documents/development/classe...   

                                              MSP R Paths  \
0       /Users/brinkley97/Documents/development/classe...   
1       /Users/brinkley97/Documents/development/classe...   
2       /Users/brinkley97/Documents/development/classe...   
3       /Users/brinkley97/Documents/development/classe...   
4       /Users/brinkley97/Documents/development/classe...   
...                                                   ...   
114818                                                NaN   
114819                                                NaN   
114820                                                NaN   
114821                                                NaN   
114822                                                NaN   

                                              MSP T Paths  
0       /Users/brinkley97/Documents/development/classe...  
1       /Users/brinkley97/Documents/development/classe...  
2       /Users/brinkley97/Documents/development/classe...  
3       /Users/brinkley97/Documents/development/classe...  
4       /Users/brinkley97/Documents/development/classe...  
...                                                   ...  
114818                                                NaN  
114819                                                NaN  
114820                                                NaN  
114821                                                NaN  
114822                                                NaN  

[114823 rows x 4 columns]

# CREMA-D

## Sample

In [ ]:
x_crema_d = extract_features(update_crema_d_paths)

In [ ]:
type(x_crema_d)

In [ ]:
import pickle

In [ ]:
sample_crema_d = BASE + CLASS_PATH + DATASET_PATH + 'inputs-sampleCremaD.txt'

# d = {'a':0,'b':1,'c':2}
with open(sample_crema_d, 'wb') as f:
    pickle.dump(x_crema_d,f)  

In [ ]:
reading_sample_crema_d = pd.read_pickle(sample_crema_d)

In [ ]:
# reading_sample_crema_d

In [ ]:
all_last_hidden_states = []
for specific_frame in range(len(reading_sample_crema_d)):
    inputs = reading_sample_crema_d[specific_frame]
    outputs = model(**inputs)
    # last_hidden_states == representation (1 and 2 with GradCam)
    last_hidden_states = outputs.last_hidden_state
    all_last_hidden_states.append(last_hidden_states)

In [ ]:
# all_last_hidden_states

In [ ]:
sample_crema_d_hidden_states = BASE + CLASS_PATH + DATASET_PATH + 'hiddenStates-sampleCremaD.txt'

d = {'a':0,'b':1,'c':2}
with open(sample_crema_d_hidden_states, 'wb') as f:
    pickle.dump(all_last_hidden_states, f)  

In [ ]:
reading_sample_crema_d_hidden_states = pd.read_pickle(sample_crema_d_hidden_states)

In [ ]:
# reading_sample_crema_d_hidden_states

In [ ]:
extracted_features_per_video = torch.empty((0, 197, 768))
save_efs = []
for specific_frame_hs in range(len(reading_sample_crema_d_hidden_states)):
    
    hs = reading_sample_crema_d_hidden_states[specific_frame_hs]
    extracted_features_per_video = torch.vstack((extracted_features_per_video, hs))
    save_efs.append(extracted_features_per_video)

In [ ]:
# save_efs

In [ ]:
sample_crema_d_extracted_features = BASE + CLASS_PATH + DATASET_PATH + 'extractedFeatures-sampleCremaD.txt'

In [ ]:

# np.savetxt(sample_crema_d_extracted_features, )
torch.save(save_efs, sample_crema_d_extracted_features)

# d = {'a':0,'b':1,'c':2}
# with open(sample_crema_d_extracted_features, 'wb') as f:
#     pickle.dump(save_efs, f)  

In [ ]:
reading_sample_crema_d_efs = pd.read_pickle(sample_crema_d_extracted_features)

In [ ]:
fe_crema_d = torch.load(sample_crema_d_extracted_features)

In [ ]:
# fe_crema_d

# MSP

In [11]:
MSP_DATASET_PATH = 'msp/videos/r_and_t_frames/'

## R

In [15]:
msp_r_paths = dataset_paths_copy['MSP R Paths']
type(msp_r_paths), msp_r_paths

(pandas.core.series.Series,
 0         /Users/brinkley97/Documents/development/classe...
 1         /Users/brinkley97/Documents/development/classe...
 2         /Users/brinkley97/Documents/development/classe...
 3         /Users/brinkley97/Documents/development/classe...
 4         /Users/brinkley97/Documents/development/classe...
                                 ...                        
 114818                                                  NaN
 114819                                                  NaN
 114820                                                  NaN
 114821                                                  NaN
 114822                                                  NaN
 Name: MSP R Paths, Length: 114823, dtype: object)

In [16]:
sub_set_msp_r_paths = msp_r_paths.loc[0:300]
sub_set_msp_r_paths

0      /Users/brinkley97/Documents/development/classe...
1      /Users/brinkley97/Documents/development/classe...
2      /Users/brinkley97/Documents/development/classe...
3      /Users/brinkley97/Documents/development/classe...
4      /Users/brinkley97/Documents/development/classe...
                             ...                        
296    /Users/brinkley97/Documents/development/classe...
297    /Users/brinkley97/Documents/development/classe...
298    /Users/brinkley97/Documents/development/classe...
299    /Users/brinkley97/Documents/development/classe...
300    /Users/brinkley97/Documents/development/classe...
Name: MSP R Paths, Length: 301, dtype: object

In [ ]:
# msp_r_df = pd.DataFrame(sub_set_msp_r_paths, columns=['MSP R Paths'])
# msp_r_df

In [24]:
x = []
    
for msp_r_path_idx in range(len(sub_set_msp_r_paths)):
    print(msp_r_path_idx)
    msp_r_path = msp_r_paths[msp_r_path_idx]
    
    specific_frame = Image.open(msp_r_path)
    # print(np.shape(specific_frame))

    inputs = processor(images=specific_frame, return_tensors="pt")
    x.append(inputs)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
# x

In [27]:
import pickle

# Input

In [30]:
sub_set_msp_r = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'inputs-mspR_subset.txt'
sub_set_msp_r

'/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/subsets_vit/inputs-mspR_subset.txt'

In [29]:
with open(sub_set_msp_r, 'wb') as f:
    pickle.dump(x, f)  

In [31]:
reading_sub_set_msp_r = pd.read_pickle(sub_set_msp_r)

In [ ]:
# reading_sub_set_msp_r

# Hidden

In [ ]:
all_last_hidden_states = []

for specific_frame in range(len(reading_sub_set_msp_r)):
    inputs = reading_sub_set_msp_r[specific_frame]
    outputs = model(**inputs)
    
    # last_hidden_states == representation (1 and 2 with GradCam)
    last_hidden_states = outputs.last_hidden_state
    all_last_hidden_states.append(last_hidden_states)

In [33]:
# 'subsets_vit/'
msp_r_hidden_states = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'hiddenStates-mspR.txt'

In [34]:
with open(msp_r_hidden_states, 'wb') as f:
    pickle.dump(all_last_hidden_states, f)  

In [ ]:
reading_msp_r_hidden_states = pd.read_pickle(msp_r_hidden_states)

In [ ]:
# reading_msp_r_hidden_states

In [ ]:
extracted_features_per_video = torch.empty((0, 197, 768))
save_efs = []
for specific_frame_hs in range(len(reading_msp_r_hidden_states)):
    
    hs = reading_msp_r_hidden_states[specific_frame_hs]
    extracted_features_per_video = torch.vstack((extracted_features_per_video, hs))
    
    save_efs.append(extracted_features_per_video)

In [ ]:
# save_efs

In [4]:
sample_msp_r_extracted_features = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'extractedFeatures-mspR_subset.txt'

In [ ]:
torch.save(save_efs, sample_msp_r_extracted_features)

In [7]:
fe_crema_d = torch.load(sample_msp_r_extracted_features)

In [9]:
# fe_crema_d